In [108]:
import cv2
import sys
import numpy as np
import os
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display
from paddleocr import PaddleOCR, draw_ocr
import pytesseract
import pandas as pd
import sys

In [109]:
def findTextRegion(img):
    region = []
 
    # 1. 查找轮廓
    contours, hierarchy = cv2.findContours(img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
 
    # 2. 筛选那些面积小的
    for i in range(len(contours)):
        cnt = contours[i]
        # 计算该轮廓的面积
        area = cv2.contourArea(cnt) 
 
        # 面积小的都筛选掉
        if(area < 1000):
            continue
 
        # 轮廓近似，作用很小
        epsilon = 0.001 * cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, epsilon, True)
 
        # 找到最小的矩形，该矩形可能有方向
        rect = cv2.minAreaRect(cnt)
        #print "rect is: "
        #print rect
 
        # box是四个点的坐标
        box = cv2.boxPoints(rect)
        box = np.int0(box)
 
        # 计算高和宽
        height = abs(box[0][1] - box[2][1])
        width = abs(box[0][0] - box[2][0])
 
        # 筛选那些太细的矩形，留下扁的
        if(height > width * 1.2):
            continue
 
        region.append(box)
 
    return region

In [110]:
def preprocess(gray):
    # 1. Sobel算子，x方向求梯度
    sobel = cv2.Sobel(gray, cv2.CV_8U, 1, 0, ksize = 3)
    # 2. 二值化
    ret, binary = cv2.threshold(sobel, 0, 255, cv2.THRESH_OTSU+cv2.THRESH_BINARY)
 
    # 3. 膨胀和腐蚀操作的核函数
    element1 = cv2.getStructuringElement(cv2.MORPH_RECT, (30, 9))
    element2 = cv2.getStructuringElement(cv2.MORPH_RECT, (24, 6))
 
    # 4. 膨胀一次，让轮廓突出
    dilation = cv2.dilate(binary, element2, iterations = 1)
 
    # 5. 腐蚀一次，去掉细节，如表格线等。注意这里去掉的是竖直的线
    erosion = cv2.erode(dilation, element1, iterations = 1)
 
    # 6. 再次膨胀，让轮廓明显一些
    dilation2 = cv2.dilate(erosion, element2, iterations = 3)
 
    # 7. 存储中间图片 
    cv2.imwrite("binary.png", binary)
    cv2.imwrite("dilation.png", dilation)
    cv2.imwrite("erosion.png", erosion)
    cv2.imwrite("dilation2.png", dilation2)
 
    return dilation2

In [111]:
def detect(img):
    # 1.  转化成灰度图
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
 
    # 2. 形态学变换的预处理，得到可以查找矩形的图片
    dilation = preprocess(gray)
 
    # 3. 查找和筛选文字区域
    region = findTextRegion(dilation)
 
    # 4. 用绿线画出这些找到的轮廓
    #for box in region:
        #cv2.drawContours(img, [box], 0, (0, 255, 0), 2)
 
    #cv2.namedWindow("img", cv2.WINDOW_NORMAL)
    #cv2.imshow("img", img)
 
    # 带轮廓的图片
    #cv2.imwrite("contours.png", img)
 
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    return(region)

In [112]:
def my_ocr(img_path):
    ocr = PaddleOCR(use_angle_cls=True, lang='ch') # need to run only once to download and load model into memory
    result = ocr.ocr(img_path, det=False)
    
    for line in result:
        print(line)
        
    return result

In [113]:
def getcor(i,coordinates):
    
    x = [] ; y = []

    for temp in coordinates[i]:
        x.append(temp[1])
        y.append(temp[0])

    xmax = max(x) ; xmin = min(x)
    ymax = max(y) ; ymin = min(y)

    result = [xmin,xmax,ymin,ymax]
    
    return result

In [114]:
def crop(img,cor):
    
    result = img[cor[0]:cor[1],cor[2]:cor[3]]
    
    return result

In [115]:
def avg_distance(coordinate):
    a = 0
    for i in range (len(coordinate)-1):
        dis1 = getcor(i,coordinate)
        dis2 = getcor(i+1,coordinate)
        a = a + (dis2[0]+dis2[1] - dis1[0] - dis1[1])
    
    return (a/2)/len(coordinate)

In [116]:
def impath(img_dir,img_name):
    
    imagePath = []
    
    for i in img_name:
        imagePath.append(img_dir + '/'+ i)
    
    return(imagePath)

In [117]:
img_dir = '/Users/ryan/OneDrive - The University of Nottingham Ningbo China/Y2/OCR Summer research/Python_Algorithm/OCR Dataset'
result_dir = '/Users/ryan/OneDrive - The University of Nottingham Ningbo China/Y2/OCR Summer research/Python_Algorithm/Result' 
img_name = sorted(os.listdir(img_dir))[2:]

In [118]:
imPath = impath(img_dir,img_name)

In [128]:
#imagePath = 'C:/Users/HW/Documents/Python Scripts/OCR/OneDrive_2022-06-23/OCR Dataset/IMG_0845.JPG'
#imagePath = 'deskewed.png'
nameindex = 0
for imagePath in imPath:
    
    if __name__ == '__main__':
        # 读取文件
        #imagePath = sys.argv[1]
        img = cv2.imread(imagePath)
        cor=detect(img)

    coordinates=cor[::-1]
    coordinates_int10=np.round(np.divide(coordinates,10))
    numberofrows=len(coordinates)

    columns=[['']]
    numofparalines=0
    dis = avg_distance(coordinates_int10)


    for i in range(numberofrows):

        if i >= 1:
            tempcor1_int10 = getcor(i-1,coordinates_int10)
            tempcor2_int10 = getcor(i,coordinates_int10)


        if i==0:
            tempcor = getcor(i,coordinates)
            crop_img = crop(img,tempcor)
            #result = [pytesseract.image_to_string (crop_img, lang='chi_sim')]
            result = [my_ocr(crop_img)[0][0]]

            if result != ['']:
                columns.append(result)
                #blockindex.append(i)
            else:
                numofparalines+=1

        elif (tempcor2_int10[0]+tempcor2_int10[1])/2 - (tempcor1_int10[0]+tempcor1_int10[1])/2> 2*dis: # should be the same as the lower one

            rowcor = [] # Use to take down the index of each row

            tempcor = getcor(i,coordinates)
            crop_img = crop(img,tempcor)
            #result = [pytesseract.image_to_string (crop_img, lang='chi_sim')]
            result = [my_ocr(crop_img)[0][0]]

            if result != ['']:
                columns.append(result)
                startrowindex = i
                rowcor.append(tempcor[2])
            else:
                numofparalines+=1


        else:
            numofparalines+=1
            tempcor1 = getcor(i-1,coordinates)
            tempcor2 = getcor(i,coordinates)

            tempcor = getcor(i,coordinates)
            crop_img = crop(img,tempcor)
            result = my_ocr(crop_img)[0][0]

            if result != ['']:
                columns[i-numofparalines].append(result)
                rowcor.append(tempcor[2])

            if i <= numberofrows-2:
                tempcor3_int10 = getcor(i+1,coordinates_int10)

                if (tempcor3_int10[0]+tempcor3_int10[1])/2 - (tempcor2_int10[0]+tempcor2_int10[1])/2> 2*dis: # should be the same as in the previous one

                    if len(rowcor) != 0:
                        
                        if len(columns)==1:
                            sortedrowcor,sortedrow = zip(*sorted(zip(rowcor,columns[-1])))
                            del columns[-1]
                            columns.append(list(sortedrow))
                            
                        else:
                            sortedrowcor,sortedrow = zip(*sorted(zip(rowcor,columns[-2])))

#                             print('cloumns[-2]')
#                             print(columns[-2])
#                             print('rowcor:')
#                             print(rowcor)
#                             print('sortedrowcor:')
#                             print(sortedrowcor)
#                             print('sortedrow:')
#                             print(sortedrow)
                            tempcolumn = columns[-1]
                            del columns[-1]
                            del columns[-1]
                            columns.append(list(sortedrow))
                            columns.append(tempcolumn)


    df=pd.DataFrame(columns)
    df.fillna('',inplace=True)

    # path="result/result.xlsx"
    # if not os.path.exists(path):
    #     os.makedirs(path)
    
    name = img_name[nameindex]
    stop = name.index('.')
    df.to_excel(name[0:stop]+'_result.xlsx',index=False)
    nameindex = nameindex + 1

[2022/07/11 12:21:34] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/Users/ryan/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_fce_box_type='poly', det_limit_side_len=960, det_limit_type='max', det_model_dir='/Users/ryan/.paddleocr/whl/det/ch/ch_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_box_type='quad', det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_polygon=False, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=None, e2e_p

('区设市限份有限公司', 0.5534780025482178)
[2022/07/11 12:21:37] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/Users/ryan/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_fce_box_type='poly', det_limit_side_len=960, det_limit_type='max', det_model_dir='/Users/ryan/.paddleocr/whl/det/ch/ch_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_box_type='quad', det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_polygon=False, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_typ

('', 0.0)
[2022/07/11 12:21:40] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/Users/ryan/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_fce_box_type='poly', det_limit_side_len=960, det_limit_type='max', det_model_dir='/Users/ryan/.paddleocr/whl/det/ch/ch_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_box_type='quad', det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_polygon=False, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=N

('', 0.0)
[2022/07/11 12:21:43] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/Users/ryan/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_fce_box_type='poly', det_limit_side_len=960, det_limit_type='max', det_model_dir='/Users/ryan/.paddleocr/whl/det/ch/ch_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_box_type='quad', det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_polygon=False, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=N

('本次余丽23B60调置3143米道', 0.6768952012062073)
[2022/07/11 12:21:46] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/Users/ryan/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_fce_box_type='poly', det_limit_side_len=960, det_limit_type='max', det_model_dir='/Users/ryan/.paddleocr/whl/det/ch/ch_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_box_type='quad', det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_polygon=False, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_l

...

('售', 0.983754575252533)
[2022/07/11 12:30:31] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/Users/ryan/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_fce_box_type='poly', det_limit_side_len=960, det_limit_type='max', det_model_dir='/Users/ryan/.paddleocr/whl/det/ch/ch_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_box_type='quad', det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_polygon=False, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', 

('万', 0.94344562292099)
[2022/07/11 12:30:34] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/Users/ryan/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_fce_box_type='poly', det_limit_side_len=960, det_limit_type='max', det_model_dir='/Users/ryan/.paddleocr/whl/det/ch/ch_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_box_type='quad', det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_polygon=False, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e

('复核：陈亚', 0.94853276014328)
[2022/07/11 12:30:38] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/Users/ryan/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_fce_box_type='poly', det_limit_side_len=960, det_limit_type='max', det_model_dir='/Users/ryan/.paddleocr/whl/det/ch/ch_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_box_type='quad', det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_polygon=False, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max

('', 0.0)
[2022/07/11 12:30:41] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/Users/ryan/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_fce_box_type='poly', det_limit_side_len=960, det_limit_type='max', det_model_dir='/Users/ryan/.paddleocr/whl/det/ch/ch_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_box_type='quad', det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_polygon=False, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=N

('', 0.0)


In [ ]:
sortedrowcor,sortedrow = zip(*sorted(zip(rowcor,columns[-2])))

In [124]:
list1 = [2,1,3]

In [123]:
list2 = ['a','b','c']
sorted(zip(list1,list2))

[(1, 'a'), (2, 'b'), (3, 'c')]

In [125]:
columns

[['', '']]